#Gradient Descent For Neural Network

##Method 1:
*Using Tensorflow*

###Problem: Predict if a person would go for having a life insurance based on age and affordability from this binary logistic regression.

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
%matplotlib inline

In [66]:
path = '/content/insurance_data.csv'
df = pd.read_csv(path)

In [67]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [68]:
df.shape

(28, 3)

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance, test_size=0.2, random_state= 25)

###Preprocessing: `Scaling`

In [70]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [71]:
X_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [72]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


###Model Building (using keras)

In [73]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 8ms/step - loss: 0.5603 - accuracy: 0.6818
Epoch 1833/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5603 - accuracy: 0.6818
Epoch 1834/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5602 - accuracy: 0.6818
Epoch 1835/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5602 - accuracy: 0.6818
Epoch 1836/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5601 - accuracy: 0.6818
Epoch 1837/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5601 - accuracy: 0.6818
Epoch 1838/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5600 - accuracy: 0.6818
Epoch 1839/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5600 - accuracy: 0.6818
Epoch 1840/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5599 - accuracy: 0.6818
Epoch 1841/5000
1/1 [==============================] - 

###Evaluation

In [74]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 222ms/step - loss: 0.3550 - accuracy: 1.0000


[0.3549776077270508, 1.0]

In [75]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 110ms/step


array([[0.7054848 ],
       [0.3556957 ],
       [0.16827832],
       [0.47801185],
       [0.7260696 ],
       [0.8294983 ]], dtype=float32)

In [76]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

Now, I need to find the updated weight & bias

In [77]:
coef, intercept = model.get_weights()

In [78]:
coef, intercept

(array([[5.060863 ],
        [1.4086521]], dtype=float32),
 array([-2.913703], dtype=float32))

**Result**: *w1* = 5.060863 , *w2* = 1.4086521, *bias* = -2.913703

##Method 2:
*Without using TF*

In [85]:
def sigmoid(x):
  import math
  return 1 / (1 + math.exp(-x))

sigmoid(18)

0.9999999847700205

In [86]:
X_test

,age,affordibility
2,47,1
10,18,1
21,26,0
11,28,1
14,49,1
9,61,1


In [87]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

prediction_function(.47, 1)

0.705484819775958

###Result: Accuracy is great. This value above matches our model prediction indeed

##Method 3:
Using simple Python

In [88]:
def sigmoid_numpy(X):
  return 1 / (1+ np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [89]:
##Loss Function
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(1-y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [95]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [96]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000, 0.4631)


Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:1.370885777917796
Epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:1.256686085009161
Epoch:2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.2977578997796538, loss:1.1629603740123435
Epoch:3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.3715094724003511, loss:1.0868323288868447
Epoch:4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.43506643026891584, loss:1.0254446442688272
Epoch:5, w1:0.9255469396815343, w2:0.8124402814952774, bias:-0.48994490058938817, loss:0.9761724007845884
Epoch:6, w1:0.9267936114129968, w2:0.8034375029757677, bias:-0.5375299543522853, loss:0.9367290394234877
Epoch:7, w1:0.93047170420295, w2:0.7983920007454487, bias:-0.5790424270894963, loss:0.9051928598611562
Epoch:8, w1:0.9361540784567942, w2:0.7965748796787705, bias:-0.6155315088627655, loss:0.8799863815606783
Epoch:9, w1:0.9434791243557357, w2:0.7973647616854131, bia

(8.270239516056172, 1.6332228396255535, -4.425953608965705)

In [97]:
coef, intercept

(array([[5.060863 ],
        [1.4086521]], dtype=float32),
 array([-2.913703], dtype=float32))

##Result: It's the same result what I've found via tensorflow

---

